# 使用步骤


1.  找到“导入配置”部分的NOTION_TOKEN和DATABASE_ID，引号内填写自己的token和id
2.  点击上方“全部运行”
3. 按照提示输入想插入notion的字段
3.  代码跑到最后一part会提示输入url，将力扣中文站的题目链接直接贴过来即可







# 一、导入

In [ ]:
!pip install notion-client requests

In [ ]:
import requests
import json
from notion_client import Client
from datetime import datetime
from urllib.parse import urlparse
from bs4 import BeautifulSoup

NOTION_TOKEN = ""
DATABASE_ID = ""

notion = Client(auth=NOTION_TOKEN)

# 二、GraphQL 提取LeetCode中文站信息

In [ ]:
# 从力扣url中精准提取slug
def extract_slug(url):
    path = urlparse(url).path
    parts = [p for p in path.split('/') if p]
    if 'problems' in parts:
        idx = parts.index('problems')
        if idx + 1 < len(parts):
            return parts[idx + 1]
    raise ValueError(f"未能从 URL 提取 slug: {url}")

# 入参是完整url，提取信息
def extract_leetcode_info_cn(url):
    slug = extract_slug(url)
    print(f"解析出的slug:{slug}")

    graphql_url = 'https://leetcode.cn/graphql/' # GraphQL API
    # HTTP 请求头
    headers = {
        'Content-Type': 'application/json',
        'Referer': f'https://leetcode.cn/problems/{slug}/description/',
        'User-Agent': 'Mozilla/5.0'
    }

    # 查询语句
    query = '''
    query questionData($titleSlug: String!) {
      question(titleSlug: $titleSlug) {
        questionId
        title
        translatedTitle
        difficulty
        translatedContent
        topicTags{
          name
          slug
        }
      }
    }
    '''

    variables = {"titleSlug": slug}
    res = requests.post(graphql_url, json={"query": query, "variables": variables}, headers=headers)
    res.raise_for_status()

    data = res.json()

    q = data['data']['question']

    if not q:
        raise ValueError(f"题目未找到，检查 URL 和 slug: {slug}")

    topics = [tag['name'] for tag in q['topicTags']]
    return {
        "name": q["translatedTitle"],
        "number": int(q["questionId"]),
        "difficulty": q["difficulty"].lower(),
        "description": q["translatedContent"],
        "slug": slug,
        "url": f"https://leetcode.cn/problems/{slug}/description/",
        "tags": topics
    }

# 三、html题目描述格式转换

In [ ]:
# 将 HTML 描述转换为Notion children 块列表
def html_to_notion_blocks(html):
  soup = BeautifulSoup(html, "html.parser")
  children = []

  for el in soup.children:
    # 跳过空行
        if not el.name:
            continue

        # <p> 转 paragraph 块
        if el.name == "p":
            text = el.get_text(separator=" ").strip()
            if text:
                children.append({
                    "object": "block",
                    "type": "paragraph",
                    "paragraph": {
                        "rich_text": [{
                            "type": "text",
                            "text": {"content": text}
                        }]
                    }
                })

        # <pre> 转 code 块
        elif el.name == "pre":
            code_text = el.get_text().strip()
            if code_text:
                children.append({
                    "object": "block",
                    "type": "code",
                    "code": {
                        "language": "plain text",
                        "rich_text": [{
                            "type": "text",
                            "text": {"content": code_text}
                        }]
                    }
                })

        # <ul> → bulleted_list_item 块
        elif el.name == "ul":
            for li in el.find_all("li"):
                li_text = li.get_text().strip()
                if li_text:
                    children.append({
                        "object": "block",
                        "type": "bulleted_list_item",
                        "bulleted_list_item": {
                            "rich_text": [{
                                "type": "text",
                                "text": {"content": li_text}
                            }]
                        }
                    })

        # <ol> → numbered_list_item 块
        elif el.name == "ol":
            for li in el.find_all("li"):
                li_text = li.get_text().strip()
                if li_text:
                    children.append({
                        "object": "block",
                        "type": "numbered_list_item",
                        "numbered_list_item": {
                            "rich_text": [{
                                "type": "text",
                                "text": {"content": li_text}
                            }]
                        }
                    })

        # <h2> → heading_2 块
        elif el.name == "h2":
            heading = el.get_text().strip()
            if heading:
                children.append({
                    "object": "block",
                    "type": "heading_2",
                    "heading_2": {
                        "rich_text": [{
                            "type": "text",
                            "text": {"content": heading}
                        }]
                    }
                })
  return children

# 四、写入Notion

In [ ]:
def add_to_notion(info):
    # 查询是否存在
    result = notion.databases.query(
        database_id = DATABASE_ID,
        filter={
            "property": "Number",
            "number":{
                "equals": info["number"]
            }
        }
    )
    today = datetime.now().strftime("%Y-%m-%d")

    # 弹出Importance
    importance = input("Importance（必背/重点/普通）: ").strip()
    importance = importance if importance else None
    # 弹出review
    review = input("Review状态（need review/first pass/second pass/third pass+）: ").strip()
    review = review if review else None
    # 弹出comments
    comments_input = input("请输入你的Comments > ").strip()
    comments = None
    if comments_input:
      comments = {
          "rich_text": [
              {"text": {"content": comments_input}}
          ]
      }

    # 完整props
    props = {
        "Problem Name": {
            "title": [{"text": {"content": info['name']}}]
        },
        "Number": {"number": info['number']},
        "Link": {"url": info['url']},
        "Difficulty": {"select": {"name": info['difficulty']}},
        "Topic": {"multi_select": [{"name": tag} for tag in info["tags"]]},
        "Importance": {"select": {"name": importance}} if importance else None,
        "Review": {"status": {"name": review}} if review else None,
        "Comments": comments
    }
    props = {k: v for k, v in props.items() if v is not None}

    # 已存在则只更新部分props
    update_props = {
        "Importance": {"select": {"name": importance}} if importance else None,
        "Review": {"status": {"name": review}} if review else None,
        "Comments": comments
    }
    update_props = {k: v for k, v in update_props.items() if v is not None}

    # 页面内部内容块
    children = [
      {
        "object": "block",
        "type": "heading_2",
        "heading_2": {
            "rich_text": [{"text": {"content": "📄 题目描述"}}]
        }
      }
    ] + html_to_notion_blocks(info['description'])

    # 分支执行
    if result["results"]:
        page_id = result["results"][0]["id"]
        notion.pages.update(
            page_id=page_id,
            properties=update_props
        )
        print(f"✅ 已存在，已更新 Review → {info['name']}")
    else:
        notion.pages.create(
            parent={"database_id": DATABASE_ID},
            properties=props,
            children=children
        )
        print(f"✅ 首次刷题，已新建 → {info['name']}")

# 五、手动输入Url

In [ ]:
url = input("✍️ 请输入要同步到Notion的LeetCode中文站URL:\n例如:https://leetcode.cn/problems/two-sum/description/\n>")
info = extract_leetcode_info_cn(url)
print("\n✅ 抓取结果:")
print(json.dumps(info, indent=2, ensure_ascii=False))

# 写入Notion
add_to_notion(info)
print("✅ 已成功写入 Notion!")